In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
directory = '/mnt/sun-temp/TEMP/PRINCE_syncing/'
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']=='860']

In [9]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
795,/mnt/sun-temp/TEMP/PRINCE_syncing/20220319_183...,860,11,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"19.03.2022, 18:38:",20220319_1838_Plate11,NaN
796,/mnt/sun-temp/TEMP/PRINCE_syncing/20220321_083...,860,11,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"21.03.2022, 08:38:",20220321_0838_Plate11,NaN
827,/mnt/sun-temp/TEMP/PRINCE_syncing/20220320_203...,860,11,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"20.03.2022, 20:38:",20220320_2038_Plate11,NaN
834,/mnt/sun-temp/TEMP/PRINCE_syncing/20220320_083...,860,11,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"20.03.2022, 08:38:",20220320_0838_Plate11,NaN
873,/mnt/sun-temp/TEMP/PRINCE_syncing/20220320_223...,860,11,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"20.03.2022, 22:38:",20220320_2238_Plate11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12844,/mnt/sun-temp/TEMP/PRINCE_syncing/20220413_174...,860,25,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"13.04.2022, 17:45:",20220413_1745_Plate25,NaN
12853,/mnt/sun-temp/TEMP/PRINCE_syncing/20220414_054...,860,25,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"14.04.2022, 05:45:",20220414_0545_Plate25,NaN
12861,/mnt/sun-temp/TEMP/PRINCE_syncing/20220414_014...,860,25,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"14.04.2022, 01:45:",20220414_0145_Plate25,NaN
12869,/mnt/sun-temp/TEMP/PRINCE_syncing/20220413_214...,860,25,'Carrot','A5','001P100N200C','Y','-',20220317,25,...,False,False,False,False,False,False,False,"13.04.2022, 21:45:",20220413_2145_Plate25,NaN


In [10]:
dir_drop = 'prince_data'


In [11]:
run_info = folders.copy()
folder_list = list(run_info['folder'])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info['unique_id'] = run_info['Plate'].astype(str) + "_" + run_info['CrossDate'].astype(str)
        line = run_info.loc[run_info['folder']==directory_name]
        id_unique = line['unique_id'].iloc[0]

        path_snap=directory+directory_name
        API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))

        path_zip =f'{os.getenv("TEMP")}{directory_name}.zip'
        path_info =f'{os.getenv("TEMP")}{directory_name}_info.json'
        line.to_json(path_info)
        zip_file(path_snap,path_zip)
        upload(API,path_zip,f'/{dir_drop}/{id_unique}/{directory_name}.zip',chunk_size=256 * 1024 * 1024)
        upload(API,path_info,f'/{dir_drop}/{id_unique}/{directory_name}_info.json',chunk_size=256 * 1024 * 1024)
        os.remove(path_info) 
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/115 [00:00<?, ?it/s]

zipping:   0%|          | 0/130 [00:00<?, ?it/s]

KeyboardInterrupt: 